# 在LCEL出现之前的遗留Chain

- Runnable
    - RunnableSerializable
        - Chain
            - LLMChain
                - ConversationChain
                - QuestionGeneratorChain
                - FlareChain
            - LLMCheckerChain
            - LLMRequestsChain
            - LLMMathChain（使用python代码执行数学计算）
            - LLMSummarizationCheckerChain
            - MapReduceChain
            - OpenAIModerationChain
            - SequentialChain
            - SimpleSequentialChain
            - APIChain
            - BaseCombineDocumentsChain
            - AnalyzeDocumentChain
            - ConstitutionalChain
            - BaseConversationalRetrievalChain
                - ConversationalRetrievalChain
                - ChatVectorDBChain
            - ElasticsearchDatabaseChain
            - NatBotChain（实现一个基于LLM的浏览器）
            - QAGenerationChain（问答对生成）
            - BaseQAWithSourcesChain
                - QAWithSourcesChain
            - BaseRetrievalQA
                - RetrievalQA
                - VectorDBQA
            - RouterChain
                - MultiRouteChain
                    - MultiRetrievalQAChain
                - EmbeddingRouterChain
                - LLMRouterChain

# 使用LCEL构建的预制 Chain

- create_stuff_documents_chain（**携带Document**的链）
- create_openai_fn_runnable（执行**OpenAI Function Calling**，在必要时选择一个函数执行）
- create_structured_output_runnable （执行**OpenAI Function Calling**，强制选择至少一个函数执行）
- load_query_constructor_runnable
- create_sql_query_chain（生成**SQL**的链）
- create_history_aware_retriever
- create_retrieval_chain（结合`create_stuff_documents_chain`构建**RAG**链）

## create_stuff_documents_chain

In [314]:
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = create_stuff_documents_chain(llm, prompt)

docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

chain.invoke({"context": docs})

"Jesse's favorite color is red\nJamal's favorite color is orange"

## create_openai_fn_runnable

In [315]:
from typing import Optional

from langchain.chains.openai_functions import create_openai_fn_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class RecordPerson(BaseModel):
    """Record some identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class RecordDog(BaseModel):
    """Record some identifying information about a dog."""

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")


llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world class algorithm for recording entities."),
        ("human", "Make calls to the relevant function to record the entities in the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)
chain = create_openai_fn_runnable([RecordPerson, RecordDog], llm, prompt)
chain.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

RecordDog(name='Harry', color='brown', fav_food='chicken')

## load_query_constructor_runnable

## create_sql_query_chain

In [328]:
# poetry add langchain langchain-community langchain-openai
# poetry add duckdb-engine
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 这段代码由于duckdb暂时不支持查询索引元数据，因此会出现警告信息
db = SQLDatabase.from_uri("duckdb:///data/langchain.duckdb")
llm = ChatOpenAI(temperature=0)
chain = create_sql_query_chain(llm, db)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/duckdb_engine/__init__.py:180: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [330]:
response = chain.invoke({"question": "我想知道文本块的长度分布，按照每1000个字一个步长分别统计"})
print(response)

SELECT (LENGTH(content) / 1000) * 1000 AS length_range, COUNT(*) AS count
FROM text_blocks
GROUP BY length_range
ORDER BY length_range
LIMIT 5;


## create_history_aware_retriever

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import create_history_aware_retriever
from langchain import hub

rephrase_prompt = hub.pull("langchain-ai/chat-langchain-rephrase")
llm = ChatOpenAI()
retriever = ...
chat_retriever_chain = create_history_aware_retriever(
    llm, retriever, rephrase_prompt
)

chain.invoke({"input": "...", "chat_history": })

## create_retrieval_chain

In [334]:
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")
documents = loader.load()

documents = CharacterTextSplitter().split_documents(documents)

embeddings = OpenAIEmbeddings()

In [344]:
import lancedb

db = lancedb.connect("data/demo.lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

search_docs = LanceDB.from_documents(documents, embeddings, connection=table)
retriever = search_docs.as_retriever()

In [ ]:
!poetry add langchainhub

In [350]:
search_docs.search(

In [348]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
llm = ChatOpenAI()
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [349]:
retrieval_chain.invoke({"input": "总统说了什么？"})

{'input': '总统说了什么？',
 'context': [Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspire